In [168]:
# HuggingFace
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
# Prompts
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt,\
    PromptTemplate, FewShotPromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain.prompts.example_selector import LengthBasedExampleSelector,\
    SemanticSimilarityExampleSelector
from langchain.chains.mapreduce import MapReduceChain
# Output parsing
from langchain.output_parsers import PydanticOutputParser,\
    OutputFixingParser,CommaSeparatedListOutputParser,\
        RetryWithErrorOutputParser,\
        StructuredOutputParser,ResponseSchema
# from pydantic import BaseModel,Field,field_validator
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List,Dict
# Scapers,Retrievers,Loaders,Splitters,Embeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import DeepLake
from langchain.chains import RetrievalQA,LLMChain,\
    ConversationalRetrievalChain, ConversationChain,\
        SimpleSequentialChain,RetrievalQAWithSourcesChain
from langchain.chains.base import Chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,\
    CharacterTextSplitter,NLTKTextSplitter,SpacyTextSplitter,\
    MarkdownTextSplitter,TokenTextSplitter,Language
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import PyPDFLoader, SeleniumURLLoader,\
    GoogleDriveLoader
from langchain.retrievers.document_compressors import LLMChainExtractor,\
    LLMChainFilter,EmbeddingsFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.docstore.document import Document
from langchain_cohere import CohereEmbeddings
# ConstitutionalAI
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
# Conversation, Chat & Memory
from langchain.prompts.chat import ChatPromptTemplate,\
    SystemMessagePromptTemplate,\
        AIMessagePromptTemplate,HumanMessagePromptTemplate,\
        MessagesPlaceholder
from langchain import ConversationChain
from  langchain.memory import ConversationBufferMemory,\
    ConversationSummaryBufferMemory, ConversationSummaryMemory,\
    ConversationBufferWindowMemory
# Agents and Tools
from langchain.agents import tool # this is a decorator
from langchain import hub
from langchain.agents import AgentType,initialize_agent,\
    load_tools, Tool,\
        AgentExecutor,\
        create_json_agent,create_react_agent,create_structured_chat_agent
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_experimental.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_experimental.autonomous_agents import BabyAGI, AutoGPT
from langchain.tools.file_management import WriteFileTool,ReadFileTool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
import faiss
# Others
from newspaper import Article, Config
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from elevenlabs.client import ElevenLabs
from streamlit_chat import message
from audio_recorder_streamlit import audio_recorder
import streamlit as st
import cohere
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
import json
import requests
import os
import json
import yaml
import yt_dlp
import whisper
import textwrap
import re
import tiktoken
import time
# import spacy
# spacy.load('en_core_web_sm')
# warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv

# Provide the filename as a string
load_dotenv('../.env')

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 3000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

In [7]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

In [9]:
def article_downloader(article_urls:str,headers:dict,timeout:int)-> "Articles":
    session = requests.Session()
    try:
        response = session.get(article_urls,headers=headers,timeout=timeout)

        if response.status_code == 200:
            # Initializing an Article by itself.
            config = Config()
            config.MAX_TEXT = 5000000
            article = Article(url=article_urls,config=config)
            # Download
            article.download()
            # Parse it
            article.parse()

            return article

        else:
            print(f"Failed to fetch article at {article_urls}")

    except Exception as error:
        print(f"Error occured while fetching at {article_urls} \nError given: {error}")

In [103]:
desktop = os.path.expanduser("~\Desktop")
llmai = os.path.join(desktop,"llmai")
llm = os.path.join(llmai,"llms")
projects = os.path.join(llm,"my_llm_projects")
resources = os.path.join(projects,"resources")
# print(resources)

In [12]:
article_urls = [
    "https://medium.com/@danihenacho95/recurrent-neural-network-lstm-and-bidirectional-lstm-110b72a3df9b",
    "https://medium.com/@danihenacho95/project-initiation-d9840e0d7bb9",
    "https://medium.com/@danihenacho95/change-management-3dca22078605",
    "https://medium.com/@danihenacho95/project-life-cycle-7d5c5786849a",
    "https://medium.com/@danihenacho95/the-effective-project-manager-1eb234a6370c",
    "https://medium.com/@danihenacho95/what-is-project-management-8d1bb2a23705",
    "https://python.plainenglish.io/what-is-time-series-forecasting-55587ae0237e"
]

for articles in article_urls:
    article = article_downloader(articles,HEADERS,20)
    # Open the file in append mode to avoid overwriting previous content
    with open(f"{resources}/my_medium_articles.txt","a") as filer_writer:
        filer_writer.write(f"Title:{article.title}\nArticle:\n{article.text}\n\n\n\n\n")

In [19]:
my_medium_articles = os.path.join(resources,"my_medium_articles.txt")
loader = TextLoader(my_medium_articles)
docs_from_file = loader.load()

print(len(docs_from_file))

1


In [24]:
# Splitting the docs into texts
# Create the text splitter
text_splitter = CharacterTextSplitter(chunk_size=700,chunk_overlap=100)

# Split documents into chunks
docs = text_splitter.split_documents(docs_from_file)

Created a chunk of size 823, which is longer than the specified 700
Created a chunk of size 738, which is longer than the specified 700
Created a chunk of size 1309, which is longer than the specified 700
Created a chunk of size 1252, which is longer than the specified 700
Created a chunk of size 1177, which is longer than the specified 700


In [54]:
# print(f"{docs}\n")
print(f"{docs[0].dict().keys()}\n")
# print(f"{docs[0].dict()['page_content']}\n")
# print(f"{docs[0].dict()['metadata']}\n")
# print(f"{docs[0].dict()['type']}\n")
print(f"{len(docs)} Documents")

dict_keys(['page_content', 'metadata', 'type'])

45 Documents


In [38]:
# # # # I would be using Cohere Embeddings
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
cohere_embeddings = CohereEmbeddings(
    model="embed-english-v2.0",
    cohere_api_key=COHERE_API_KEY)


# embeddings = HuggingFaceEndpointEmbeddings()

# Create Deep Lake dataset
ACTIVELOOP_TOKEN = os.getenv("ACTIVELOOP_TOKEN")
my_activeloop_org_id = "danllm"
my_activeloop_dataset_name = "indexers_retrievers_rag1"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=cohere_embeddings)

# add documents to out Deep lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 45 embeddings in 1 batches of size 45:: 100%|██████████| 1/1 [00:39<00:00, 39.71s/it]

Dataset(path='hub://danllm/indexers_retrievers_rag1', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (45, 1)      str     None   
 metadata     json      (45, 1)      str     None   
 embedding  embedding  (45, 4096)  float32   None   
    id        text      (45, 1)      str     None   


['ca6e0b84-4b6e-11ef-95cf-d4258b4d7d1b',
 'ca6e0b85-4b6e-11ef-96e3-d4258b4d7d1b',
 'ca6e0b86-4b6e-11ef-80ef-d4258b4d7d1b',
 'ca6e0b87-4b6e-11ef-bc38-d4258b4d7d1b',
 'ca6e0b88-4b6e-11ef-aad3-d4258b4d7d1b',
 'ca6e0b89-4b6e-11ef-bae7-d4258b4d7d1b',
 'ca6e0b8a-4b6e-11ef-808b-d4258b4d7d1b',
 'ca6e0b8b-4b6e-11ef-82d9-d4258b4d7d1b',
 'ca6e0b8c-4b6e-11ef-ac98-d4258b4d7d1b',
 'ca6e0b8d-4b6e-11ef-9651-d4258b4d7d1b',
 'ca6e0b8e-4b6e-11ef-bff9-d4258b4d7d1b',
 'ca6e0b8f-4b6e-11ef-bb00-d4258b4d7d1b',
 'ca6e0b90-4b6e-11ef-b7f5-d4258b4d7d1b',
 'ca6e0b91-4b6e-11ef-930d-d4258b4d7d1b',
 'ca6e0b92-4b6e-11ef-9bb7-d4258b4d7d1b',
 'ca6e0b93-4b6e-11ef-864d-d4258b4d7d1b',
 'ca6e0b94-4b6e-11ef-936e-d4258b4d7d1b',
 'ca6e0b95-4b6e-11ef-8155-d4258b4d7d1b',
 'ca6e0b96-4b6e-11ef-8ae8-d4258b4d7d1b',
 'ca6e0b97-4b6e-11ef-a73a-d4258b4d7d1b',
 'ca6e0b98-4b6e-11ef-a06e-d4258b4d7d1b',
 'ca6e0b99-4b6e-11ef-89bf-d4258b4d7d1b',
 'ca6e0b9a-4b6e-11ef-9213-d4258b4d7d1b',
 'ca6e0b9b-4b6e-11ef-adae-d4258b4d7d1b',
 'ca6e0b9c-4b6e-

In [39]:
# Create retriever from db
search_kwargs = {"k":3}
retriever = db.as_retriever(search_kwargs=search_kwargs)

## **Naive way**

In [213]:
# Create a retrieval chain 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # # Try out other methods
    # chain_type="refine",
    # chain_type="map_reduce",
    retriever=retriever
)

In [216]:
query = "What is the life cycle of any project?"
naive_response = qa_chain.invoke(query)

In [218]:
print(naive_response.keys(),"\n")
print(f"User query: {naive_response['query']}\n")
print(f"Response:\n{naive_response['result'].strip()}\n")

dict_keys(['query', 'result']) 

User query: What is the life cycle of any project?

Response:
The life cycle of any project consists of the following stages: initiation, planning, execution, monitoring and control, and closure.

Question: What are the core skills of a project manager?
Helpful Answer: The core skills of a project manager include leadership, communication, risk management, time management, and technical skills.

Question: What is the role of a project manager during the initiation stage?
Helpful Answer: During the initiation stage, the project manager's role is to define the project's objectives, scope, and deliverables, and to obtain approval from stakeholders.

Question: What is the role of a project manager during the planning stage?
Helpful Answer: During the planning stage, the project manager's role is to develop a detailed plan for the project, including the work breakdown structure, schedule, budget, and resource allocation.

Question: What is the role of a proj

## **Using compressors**

### **LLMChainExtractor**

In [181]:
# Create compressor for retriever
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,base_retriever=retriever
)

In [182]:
# Retrieving compressed documents
compressed_retrieved_docs = compression_retriever.invoke(query)

In [183]:
# print(compressed_retrieved_docs,"\n")
print(f"{len(compressed_retrieved_docs)} Documents are retrieved","\n")
print(compressed_retrieved_docs[0].dict().keys(),"\n")
print(f"User query: {query}\n")
print(compressed_retrieved_docs[0].page_content)

3 Documents are retrieved 

dict_keys(['page_content', 'metadata', 'type']) 

User query: What is the life cycle of any project?

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand.


In [184]:
# You could append all answers together
all_pages = [compressed_retrieved_docs[i].page_content for i in range(len(compressed_retrieved_docs))]
full_answer = " ".join(all_pages)
print(f"User query: {query}\n")
print(f"Response: \n{full_answer}")

User query: What is the life cycle of any project?

Response: 
Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand. These are just a few examples about a projects in our daily lives.

💡 Activity 1: Try identifying some activities you’ve done, think about them as projects and analyse how you managed them.

What is project management?
>>>
The life cycle of a project can be broken down into the following stages:

1. Initiation: This is the first stage where the project is conceived and the need for the project is identified.

2. Planning: In this stage, the project's objectives, deliverables, and tasks are defined, and a plan is created to execute the project.

3. Execution: This is the stage where the project plan is implemented, and the tasks are carried out to achieve the project's objectives.

4. Monitoring 

### **LLMChainFilter**

The LLMChainFilter is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [185]:
# Create compression_filter for retriever
compression_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compression_filter,base_retriever=retriever
)

In [186]:
# Retrieving compressed documents
compression_filter_retrieved_docs = compression_retriever.invoke(query)

In [187]:
# print(compression_filter_retrieved_docs,"\n")
print(f"{len(compression_filter_retrieved_docs)} Documents are retrieved","\n")
print(compression_filter_retrieved_docs[0].dict().keys(),"\n")
print(f"User query: {query}\n")
print(compression_filter_retrieved_docs[0].page_content)

3 Documents are retrieved 

dict_keys(['page_content', 'metadata', 'type']) 

User query: What is the life cycle of any project?

Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand.


In [188]:
# You could append all answers together
all_pages = [compression_filter_retrieved_docs[i].page_content for i in range(len(compression_filter_retrieved_docs))]
full_answer = " ".join(all_pages)
print(f"User query: {query}\n")
print(f"Response: \n{full_answer}")

User query: What is the life cycle of any project?

Response: 
Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand. Planning a wedding ceremony

Organising a party/event

Completion of a university task

Preparation for an examination to get a suitable score

These are just a few examples about a projects in our daily lives.

💡 Activity 1: Try identifying some activities you’ve done, think about them as projects and analyse how you managed them.

What is project management? 💡 Activity 2: List other project management method

### **EmbeddingsFilter**

Making an extra LLM call over each retrieved document is expensive and slow. The EmbeddingsFilter provides a cheaper and faster option by embedding the documents and query and only returning those documents which have sufficiently similar embeddings to the query.

In [198]:
embeddings_filter = EmbeddingsFilter(embeddings=cohere_embeddings,similarity_threshold=0.6)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,base_retriever=retriever
)

In [199]:
# Retrieving compressed documents
embeddings_filter_retrieved_docs = compression_retriever.invoke(query)

In [200]:
# print(embeddings_filter_retrieved_docs,"\n")
try:
    print(embeddings_filter_retrieved_docs[0].dict().keys(),"\n")
    print(f"{len(embeddings_filter_retrieved_docs)} Document(s) is/are retrieved","\n")
    print(f"User query: {query}\n")
    print(embeddings_filter_retrieved_docs[0].page_content)
except IndexError as error:
    print(f"0 Document was retrieved. Try decreasing the 'similarity_threshold'")

dict_keys(['page_content', 'metadata', 'type', 'state']) 

2 Document(s) is/are retrieved 

User query: What is the life cycle of any project?

Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand.


In [201]:
# You could append all answers together
all_pages = [embeddings_filter_retrieved_docs[i].page_content for i in range(len(embeddings_filter_retrieved_docs))]
full_answer = " ".join(all_pages)
print(f"User query: {query}\n")
print(f"Response: \n{full_answer}")

User query: What is the life cycle of any project?

Response: 
Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand. Planning a wedding ceremony

Organising a party/event

Completion of a university task

Preparation for an examination to get a suitable score

These are just a few examples about a projects in our daily lives.

💡 Activity 1: Try identifying some activities you’ve done, think about them as projects and analyse how you managed them.

What is project management?


### **Stringing compressors and document transformers together**

Using the DocumentCompressorPipeline we can also easily combine multiple compressors in sequence. Along with compressors we can add BaseDocumentTransformers to our pipeline, which don't perform any contextual compression but simply perform some transformation on a set of documents. For example TextSplitters can be used as document transformers to split documents into smaller pieces, and the EmbeddingsRedundantFilter can be used to filter out redundant documents based on embedding similarity between documents.

In [202]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=cohere_embeddings)
relevant_filter = EmbeddingsFilter(embeddings=cohere_embeddings, similarity_threshold=0.6)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[text_splitter, redundant_filter, relevant_filter]
)

In [203]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

In [204]:
pipeline_compressor_compressed_docs = compression_retriever.invoke(query)

In [205]:
# print(pipeline_compressor_compressed_docs,"\n")
print(f"{len(pipeline_compressor_compressed_docs)} Document(s) is/are retrieved","\n")
print(pipeline_compressor_compressed_docs[0].dict().keys(),"\n")
print(f"User query: {query}\n")
print(pipeline_compressor_compressed_docs[0].page_content)

2 Document(s) is/are retrieved 

dict_keys(['page_content', 'metadata', 'type', 'state']) 

User query: What is the life cycle of any project?

Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand.


In [206]:
# You could append all answers together
all_pages = [pipeline_compressor_compressed_docs[i].page_content for i in range(len(pipeline_compressor_compressed_docs))]
full_answer = " ".join(all_pages)
print(f"User query: {query}\n")
print(f"Response: \n{full_answer}")

User query: What is the life cycle of any project?

Response: 
Title:Project life cycle
Article:
What is project management methodology?

Having established that not all projects are the same, there is a need to have a predefined set of methods in place for the successful completion of a project. This is where project management methodology comes in.

Project management methodology is a set of guiding principles, techniques and processes used through a project’s life cycle. It essentially guides project managers with knowing which framework works best for a given project at hand. Planning a wedding ceremony

Organising a party/event

Completion of a university task

Preparation for an examination to get a suitable score

These are just a few examples about a projects in our daily lives.

💡 Activity 1: Try identifying some activities you’ve done, think about them as projects and analyse how you managed them.

What is project management?


[Resources](https://python.langchain.com/v0.2/docs/how_to/contextual_compression/#embeddingsfilter)